In [1]:
import os
import sys
import glob
import json
import tqdm
import shutil

import numpy as np
import pandas as pd
import nibabel as nib

sys.path.append(os.path.join("..", "..", ".."))
from tta_uia_segmentation.src.preprocessing.utils import get_filepaths_raw

## Extract the bias corrected TOF scans

In [22]:
output_dir = '../../../../data/preprocessed/0_bias_corrected/Lausanne'

scans_dict = get_filepaths_raw(
    path_to_tof_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/N4_bias_field_corrected',
    fp_pattern_tof=['*', '*', '*', '*angio_N4bfc_mask.nii.gz'],
    path_to_seg_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/skull_stripped_and_aneurysm_mask',
    fp_pattern_seg=['*', '*', '*', '*Lesion_1_mask.nii.gz'],
    level_of_dir_with_id=-4,
    every_scan_has_seg=False    
)

scans_dict_2 = get_filepaths_raw(
    path_to_tof_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/N4_bias_field_corrected',
    fp_pattern_tof=['*', '*', '*', '*angio_N4bfc_mask.nii.gz'],
    path_to_seg_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/skull_stripped_and_aneurysm_mask',
    fp_pattern_seg=['*', '*', '*', '*Lesion_2_mask.nii.gz'],
    level_of_dir_with_id=-4,
    every_scan_has_seg=False    
)

scans_dict_3 = get_filepaths_raw(
    path_to_tof_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/N4_bias_field_corrected',
    fp_pattern_tof=['*', '*', '*', '*angio_N4bfc_mask.nii.gz'],
    path_to_seg_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/skull_stripped_and_aneurysm_mask',
    fp_pattern_seg=['*', '*', '*', '*Lesion_3_mask.nii.gz'],
    level_of_dir_with_id=-4,
    every_scan_has_seg=False    
)

scans_dict_4 = get_filepaths_raw(
    path_to_tof_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/N4_bias_field_corrected',
    fp_pattern_tof=['*', '*', '*', '*angio_N4bfc_mask.nii.gz'],
    path_to_seg_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/skull_stripped_and_aneurysm_mask',
    fp_pattern_seg=['*', '*', '*', '*Lesion_4_mask.nii.gz'],
    level_of_dir_with_id=-4,
    every_scan_has_seg=False    
)

In [33]:
for id, info in scans_dict_3.items():
    if 'seg' in info:
        print(id)

sub-452


In [25]:
scan_dict_complete = dict()
for id, info in scans_dict.items():
    scan_dict_complete[id] = {}
    scan_dict_complete[id]['tof'] = info['tof']
    if 'seg' in info:
        scan_dict_complete[id]['seg'] = [info['seg']]
    
    for scan_d_i in [scans_dict_2, scans_dict_3, scans_dict_4]:
        if 'seg' in scan_d_i[id]:
            print(id)
            scan_dict_complete[id]['seg'].append(scan_d_i[id]['seg'])        

sub-458
sub-481
sub-480
sub-452
sub-452
sub-464
sub-487
sub-486
sub-457


In [32]:
for scan_id, scans in tqdm.tqdm(scan_dict_complete.items()):
    scan_dir_path = os.path.join(output_dir, scan_id)
    os.makedirs(scan_dir_path, exist_ok=True)
    
    # copy new_path_tof to output_dir_tof
    new_path_tof = os.path.join(scan_dir_path, f'{scan_id}_tof.nii.gz')
    shutil.copy(scans['tof'], new_path_tof)

    # copy new_path_aneurysm to output_dir_label
    if 'seg' in scans:
        # Load nifti file of all the scans and collapse them into a single mask
        segs = []
        for i, seg in enumerate(scans['seg']):
            seg_nib = nib.load(seg)
            if i == 0:
                header = seg_nib.header
                affine = seg_nib.affine
                
            segs.append(seg_nib.get_fdata())
            
        # Collapse them into single mask
        segs = np.array(segs)
        segs = np.sum(segs, axis=0)
        segs[segs > 0] = 1
        segs = nib.Nifti1Image(segs, affine, header)
        new_path_aneurysm = os.path.join(scan_dir_path, f'{scan_id}_seg.nii.gz') 
        nib.save(segs, new_path_aneurysm)
    


100%|██████████| 37/37 [01:40<00:00,  2.70s/it]


In [30]:
seg

'/'

In [16]:
scans

{'tof': '/scratch_net/biwidl319/jbermeo/data/raw/Lausanne/N4_bias_field_corrected/sub-482/ses-20140210/anat/sub-482_ses-20140210_desc-angio_N4bfc_mask.nii.gz'}

## Do it for the healthy patient scans

In [4]:
output_dir = '../../../../data/preprocessed/0_bias_corrected/Lausanne_healthy_subjects'

scans_dict = get_filepaths_raw(
    path_to_tof_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne_healthy_subjects/N4_bias_field_corrected',
    fp_pattern_tof=['*', '*', '*', '*angio_N4bfc_mask.nii.gz'],
    path_to_seg_dir='/scratch_net/biwidl319/jbermeo/data/raw/Lausanne_healthy_subjects/manual_masks',
    fp_pattern_seg=['*', '*', '*', '*Lesion_1_mask.nii.gz'],
    level_of_dir_with_id=-4,
    every_scan_has_seg=False    
)

In [6]:
for scan_id, scans in tqdm.tqdm(scans_dict.items()):
    scan_dir_path = os.path.join(output_dir, scan_id)
    os.makedirs(scan_dir_path, exist_ok=True)
    
    # copy new_path_tof to output_dir_tof
    new_path_tof = os.path.join(scan_dir_path, f'{scan_id}_tof.nii.gz')
    shutil.copy(scans['tof'], new_path_tof)

    # copy new_path_aneurysm to output_dir_label
    if 'seg' in scans:
        # Load nifti file of all the scans and collapse them into a single mask
        segs = []
        for i, seg in enumerate(scans['seg']):
            seg_nib = nib.load(seg)
            if i == 0:
                header = seg_nib.header
                affine = seg_nib.affine
                
            segs.append(seg_nib.get_fdata())
            
        # Collapse them into single mask
        segs = np.array(segs)
        segs = np.sum(segs, axis=0)
        segs[segs > 0] = 1
        segs = nib.Nifti1Image(segs, affine, header)
        new_path_aneurysm = os.path.join(scan_dir_path, f'{scan_id}_seg.nii.gz') 
        nib.save(segs, new_path_aneurysm)
    

100%|██████████| 19/19 [00:00<00:00, 28.51it/s]
